## Po poklicih

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA
library(FSA)       # For Dunn test after Kruskal-Wallis
library(ggsignif)  # For adding significance annotations
library(knitr)	   # For copyable tables

# Import the .xlsx file
file_path <- "path"
data <- read_excel(file_path, range = "A2:E104", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define new groups by profession
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% 1:6 ~ "psihiater",
    row_number() %in% c(7:10, 23) ~ "psiholog",
    row_number() %in% 11:14 ~ "klinični psiholog",
    row_number() %in% c(15:22, 24:26) ~ "delov. terapevt",
    row_number() %in% 27:103 ~ "psihoterapevt",
    TRUE ~ NA_character_
  )))


# Shapiro-Wilk Test
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene’s Test
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Function to get significance labels
get_sig_label <- function(p) {
  if (is.na(p)) return(list(label = "n.s.", star = ""))
  if (p < 0.001) return(list(label = "p < .001", star = "***"))
  if (p < 0.01)  return(list(label = "p < .01",  star = "**"))
  if (p < 0.05)  return(list(label = "p < .05",  star = "*"))
  return(list(label = "n.s.", star = ""))
}

# Save plot function with significance annotations
save_plot <- function(metric, test_type, p_value, comparisons_df = NULL) {
  sig_info <- get_sig_label(p_value)

  base_plot <- ggplot(data, aes(x = Group, y = .data[[metric]])) +
    geom_boxplot() +
    labs(
      title = paste0(metric, " po poklicih - ", test_type, " (", sig_info$label, " ", sig_info$star, ")"),
      y = metric
    ) +
    theme_minimal()

  if (!is.null(comparisons_df) && nrow(comparisons_df) > 0) {
    base_plot <- base_plot +
      geom_signif(
        comparisons = comparisons_df$comparisons,
        annotations = comparisons_df$sig,
        y_position = comparisons_df$y_pos,
        tip_length = 0.01,
        textsize = 4
      )
  }

  ggsave(paste0("poklici_", metric, "_", test_type, ".png"), plot = base_plot, width = 8, height = 6, dpi = 300)
}


# Main loop
metrics <- c("Min", "Max", "Avg", "SD")

if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  for (metric in metrics) {
    anova_res <- aov(reformulate("Group", metric), data = data)
    posthoc <- TukeyHSD(anova_res)[[1]]
    p_main <- summary(anova_res)[[1]][["Pr(>F)"]][1]

    comparisons_df <- data.frame(
      group1 = sub("-.*", "", rownames(posthoc)),
      group2 = sub(".*-", "", rownames(posthoc)),
      p = posthoc[,"p adj"],
      stringsAsFactors = FALSE
    ) %>%
      mutate(
        sig_info = lapply(p, get_sig_label),
        sig = sapply(sig_info, function(x) x$star),
        sig_label = sapply(sig_info, function(x) x$label),
        comparisons = Map(c, group1, group2),
        y_pos = max(data[[metric]], na.rm = TRUE) * 1.05 + 
                seq(0, length.out = n(), by = 0.05 * max(data[[metric]]))
      ) %>%
      filter(sig != "")

    print(summary(anova_res))

    if (nrow(comparisons_df) > 0) {
      cat(paste0("\nTukey HSD post hoc test for ", metric, "\n\n"))
      out_df <- data.frame(
        Comparison = paste(comparisons_df$group1, "-", comparisons_df$group2),
        Z = NA,
        P.unadj = NA,
        P.adj = paste0(comparisons_df$sig_label, " ", comparisons_df$sig),
        stringsAsFactors = FALSE
      )
      print(kable(out_df, format = "simple", align = "lrr"))
      write.table(out_df, "clipboard", sep = "\t", row.names = FALSE)
    }

    save_plot(metric, "ANOVA", p_main, comparisons_df)
  }

} else {
  for (metric in metrics) {
    kw_res <- kruskal.test(reformulate("Group", metric), data = data)
    dunn_res <- dunnTest(data[[metric]], data$Group, method = "bonferroni")
    dunn_df <- dunn_res$res
    p_main <- kw_res$p.value

    raw_df <- data.frame(
      group1 = sapply(strsplit(dunn_df$Comparison, " - "), `[`, 1),
      group2 = sapply(strsplit(dunn_df$Comparison, " - "), `[`, 2),
      Z = dunn_df$Z,
      p = dunn_df$P.adj,
      stringsAsFactors = FALSE
    )

    comparisons_df <- raw_df %>%
      mutate(
        sig_info = lapply(p, get_sig_label),
        sig = sapply(sig_info, function(x) x$star),
        sig_label = sapply(sig_info, function(x) x$label),
        comparisons = Map(c, group1, group2),
        y_pos = max(data[[metric]], na.rm = TRUE) * 1.05 + 
                seq(0, length.out = n(), by = 0.05 * max(data[[metric]]))
      ) %>%
      filter(sig != "")

    print(kw_res)

    if (nrow(comparisons_df) > 0) {
      cat(paste0("\nKruskal-Wallis post hoc test for ", metric, "\n\n"))
      out_df <- data.frame(
        Comparison = paste(comparisons_df$group1, "-", comparisons_df$group2),
        Z = comparisons_df$Z,
        `P.adj` = paste0(comparisons_df$sig_label, " ", comparisons_df$sig),
        stringsAsFactors = FALSE
      )
      print(kable(out_df, format = "simple", align = "lrr"))
      write.table(out_df, "clipboard", sep = "\t", row.names = FALSE)  
    }

    save_plot(metric, "Kruskal_Wallis", p_main, comparisons_df)
  }
}


## Po sektorjih

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA
library(FSA)       # For Dunn test after Kruskal-Wallis
library(ggsignif)  # For adding significance annotations
library(knitr)	   # For copyable tables

# Import the .xlsx file
file_path <- "path"
data <- read_excel(file_path, range = "A2:E104", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:22) ~ "zdravstvo",
    row_number() %in% c(23:26) ~ "soc_varstvo",
    TRUE ~ "psihoterapija"
  )))

# Shapiro-Wilk Test
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene’s Test
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Function to get significance labels
get_sig_label <- function(p) {
  if (is.na(p)) return(list(label = "n.s.", star = ""))
  if (p < 0.001) return(list(label = "p < .001", star = "***"))
  if (p < 0.01)  return(list(label = "p < .01",  star = "**"))
  if (p < 0.05)  return(list(label = "p < .05",  star = "*"))
  return(list(label = "n.s.", star = ""))
}

# Save plot function with significance annotations
save_plot <- function(metric, test_type, p_value, comparisons_df = NULL) {
  sig_info <- get_sig_label(p_value)

  base_plot <- ggplot(data, aes(x = Group, y = .data[[metric]])) +
    geom_boxplot() +
    labs(
      title = paste0(metric, " po sektorjih - ", test_type, " (", sig_info$label, " ", sig_info$star, ")"),
      y = metric
    ) +
    theme_minimal()

  if (!is.null(comparisons_df) && nrow(comparisons_df) > 0) {
    base_plot <- base_plot +
      geom_signif(
        comparisons = comparisons_df$comparisons,
        annotations = comparisons_df$sig,
        y_position = comparisons_df$y_pos,
        tip_length = 0.01,
        textsize = 4
      )
  }

  ggsave(paste0("po sektorjih_", metric, "_", test_type, ".png"), plot = base_plot, width = 8, height = 6, dpi = 300)
}


# Main loop
metrics <- c("Min", "Max", "Avg", "SD")

if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  for (metric in metrics) {
    anova_res <- aov(reformulate("Group", metric), data = data)
    posthoc <- TukeyHSD(anova_res)[[1]]
    p_main <- summary(anova_res)[[1]][["Pr(>F)"]][1]

    comparisons_df <- data.frame(
      group1 = sub("-.*", "", rownames(posthoc)),
      group2 = sub(".*-", "", rownames(posthoc)),
      p = posthoc[,"p adj"],
      stringsAsFactors = FALSE
    ) %>%
      mutate(
        sig_info = lapply(p, get_sig_label),
        sig = sapply(sig_info, function(x) x$star),
        sig_label = sapply(sig_info, function(x) x$label),
        comparisons = Map(c, group1, group2),
        y_pos = max(data[[metric]], na.rm = TRUE) * 1.05 + 
                seq(0, length.out = n(), by = 0.05 * max(data[[metric]]))
      ) %>%
      filter(sig != "")

    print(summary(anova_res))

    if (nrow(comparisons_df) > 0) {
      cat(paste0("\nTukey HSD post hoc test for ", metric, "\n\n"))
      out_df <- data.frame(
        Comparison = paste(comparisons_df$group1, "-", comparisons_df$group2),
        Z = NA,
        P.unadj = NA,
        P.adj = paste0(comparisons_df$sig_label, " ", comparisons_df$sig),
        stringsAsFactors = FALSE
      )
      print(out_df, row.names = FALSE)
    }

    save_plot(metric, "ANOVA", p_main, comparisons_df)
  }

} else {
  for (metric in metrics) {
    kw_res <- kruskal.test(reformulate("Group", metric), data = data)
    dunn_res <- dunnTest(data[[metric]], data$Group, method = "bonferroni")
    dunn_df <- dunn_res$res
    p_main <- kw_res$p.value

    raw_df <- data.frame(
      group1 = sapply(strsplit(dunn_df$Comparison, " - "), `[`, 1),
      group2 = sapply(strsplit(dunn_df$Comparison, " - "), `[`, 2),
      Z = dunn_df$Z,
      p = dunn_df$P.adj,
      stringsAsFactors = FALSE
    )

    comparisons_df <- raw_df %>%
      mutate(
        sig_info = lapply(p, get_sig_label),
        sig = sapply(sig_info, function(x) x$star),
        sig_label = sapply(sig_info, function(x) x$label),
        comparisons = Map(c, group1, group2),
        y_pos = max(data[[metric]], na.rm = TRUE) * 1.05 + 
                seq(0, length.out = n(), by = 0.05 * max(data[[metric]]))
      ) %>%
      filter(sig != "")

    print(kw_res)

    if (nrow(comparisons_df) > 0) {
      cat(paste0("\nKruskal-Wallis post hoc test for ", metric, "\n\n"))
      out_df <- data.frame(
        Comparison = paste(comparisons_df$group1, "-", comparisons_df$group2),
        Z = comparisons_df$Z,
        `P.adj` = paste0(comparisons_df$sig_label, " ", comparisons_df$sig),
        stringsAsFactors = FALSE
      )
      print(kable(out_df, format = "simple", align = "lrr"))
      write.table(out_df, "clipboard", sep = "\t", row.names = FALSE)  
    }

    save_plot(metric, "Kruskal_Wallis", p_main, comparisons_df)
  }
}


## Po poklicih (brez psihiatra)

In [ ]:
return(data)

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA
library(FSA)       # For Dunn test after Kruskal-Wallis

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\psihoter_esej2.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E101", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:3, 9:11) ~ "psiholog",
    row_number() %in% c(4:7, 12:14) ~ "klin_psiholog",
    row_number() %in% c(24:100) ~ "psihoterapevt",
    TRUE ~ "delov_terapevt"
  )))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Plot and save function
save_plot <- function(metric, test_type, p_value) {
  plot <- ggplot(data, aes(x = Group, y = .data[[metric]])) +
    geom_boxplot() +
    labs(title = paste(metric, "by Group (", test_type, " p =", round(p_value, 15), ")"))
  
  ggsave(filename = paste0(metric, "_", test_type, ".png"), plot = plot, width = 8, height = 6, dpi = 300)
}

# Perform ANOVA if normality and homogeneity assumptions are met
metrics <- c("Min", "Max", "Avg", "SD")
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  for (metric in metrics) {
    # Perform ANOVA and Tukey post-hoc tests
    anova_res <- aov(reformulate("Group", metric), data = data)
    posthoc_res <- TukeyHSD(anova_res)
    
    # Print and plot ANOVA results
    print(summary(anova_res))
    print(posthoc_res)
    save_plot(metric, "ANOVA", summary(anova_res)[[1]][["Pr(>F)"]][1])
  }
} else {
  # Perform Kruskal-Wallis and Dunn post-hoc tests if assumptions are not met
  for (metric in metrics) {
    kw_res <- kruskal.test(reformulate("Group", metric), data = data)
    dunn_res <- dunnTest(data[[metric]], data$Group, method = "bonferroni")
    
    # Print and plot Kruskal-Wallis results
    print(kw_res)
    print(dunn_res$res)
    save_plot(metric, "Kruskal_Wallis_poklici", kw_res$p.value)
  }
}


## S psihoterapevti in psihiatri

In [ ]:
# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\psihoter_esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E102", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:9) ~ "zdravstvo",
    row_number() %in% c(10:24) ~ "soc_varstvo",
    TRUE ~ "psihoterapija"
  )))

return(data)

In [ ]:
kruskal_test <- kruskal.test(Avg ~ Group, data = data)
print(kruskal_test)

library(FSA)
dunn_test_results <- dunnTest(Avg ~ Group, data = data, method = "bonferroni")
print(dunn_test_results)

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA
library(FSA)       # For Dunn test after Kruskal-Wallis

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\psihoter_esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E102", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:9) ~ "zdravstvo",
    row_number() %in% c(10:24) ~ "soc_varstvo",
    TRUE ~ "psihoterapija"
  )))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Plot and save function
save_plot <- function(metric, test_type, p_value) {
  plot <- ggplot(data, aes(x = Group, y = .data[[metric]])) +
    geom_boxplot() +
    labs(title = paste(metric, "by Group (", test_type, " p =", round(p_value, 15), ")"))
  
  ggsave(filename = paste0(metric, "_", test_type, ".png"), plot = plot, width = 8, height = 6, dpi = 300)
}

# Perform ANOVA if normality and homogeneity assumptions are met
metrics <- c("Min", "Max", "Avg", "SD")
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  for (metric in metrics) {
    # Perform ANOVA and Tukey post-hoc tests
    anova_res <- aov(reformulate("Group", metric), data = data)
    posthoc_res <- TukeyHSD(anova_res)
    
    # Print and plot ANOVA results
    print(summary(anova_res))
    print(posthoc_res)
    save_plot(metric, "ANOVA", summary(anova_res)[[1]][["Pr(>F)"]][1])
  }
} else {
  # Perform Kruskal-Wallis and Dunn post-hoc tests if assumptions are not met
  for (metric in metrics) {
    kw_res <- kruskal.test(reformulate("Group", metric), data = data)
    dunn_res <- dunnTest(data[[metric]], data$Group, method = "bonferroni")
    
    # Print and plot Kruskal-Wallis results
    print(kw_res)
    print(dunn_res$res)
    save_plot(metric, "Kruskal_Wallis", kw_res$p.value)
  }
}


## Brez psihiatra

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)


# Define groups for analysis
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esejbrez.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E24", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 9 ~ "zdravstvo",
    row_number() > 9 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() %in% c(1:3, 9:11) ~ "psiholog",
    row_number() %in% c(4:7, 12:14) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Display the modified data
return(data)


In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esejbrez.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E24", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 8 ~ "zdravstvo",
    row_number() > 8 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() %in% c(1:3, 9:11) ~ "psiholog",
    row_number() %in% c(4:7, 12:14) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Function to save plots with a specified filename
save_plot <- function(plot, filename, width = 8, height = 6, dpi = 300) {
  ggsave(filename = filename, plot = plot, width = width, height = height, dpi = dpi)
}

# Perform t-tests and ANOVA if criteria met (normality and variance)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)

  # Post-hoc tests for ANOVA
  posthoc_min <- TukeyHSD(anova_min)
  posthoc_max <- TukeyHSD(anova_max)
  posthoc_avg <- TukeyHSD(anova_avg)
  posthoc_sd <- TukeyHSD(anova_sd)

  # Print ANOVA and post-hoc results
  print(summary(anova_min)); print(posthoc_min)
  print(summary(anova_max)); print(posthoc_max)
  print(summary(anova_avg)); print(posthoc_avg)
  print(summary(anova_sd)); print(posthoc_sd)

  # List of plots with titles
  plots <- list(
    ggplot(data, aes(x = Group, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by zdravstvo and soc. varstvo_brez (p =", round(t_min$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by zdravstvo and soc. varstvo_brez (p =", round(t_max$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by zdravstvo and soc. varstvo_brez (p =", round(t_avg$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by zdravstvo and soc. varstvo_brez (p =", round(t_sd$p.value, 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by poklic_brez (ANOVA p =", round(summary(anova_min)[[1]][["Pr(>F)"]][1], 5), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by poklic_brez (ANOVA p =", round(summary(anova_max)[[1]][["Pr(>F)"]][1], 5), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by poklic_brez (ANOVA p =", round(summary(anova_avg)[[1]][["Pr(>F)"]][1], 5), ")")),
    
    ggplot(data, aes(x = SubGroup, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by poklic_brez (ANOVA p =", round(summary(anova_sd)[[1]][["Pr(>F)"]][1], 5), ")"))
  )
  
  # Filenames for each plot
  plot_filenames <- c("Min_by_zdravstvo_and_socvarstvo_brez.png", "Max_by_zdravstvo_and_socvarstvo_brez.png", "Avg_by_zdravstvo_and_socvarstvo_brez.png", "SD_by_zdravstvo_and_socvarstvo_brez.png",
                      "Min_by_poklic_brez.png", "Max_by_poklic_brez.png", "Avg_by_poklic_brez.png", "SD_by_poklic_brez.png")
  
  # Save each plot with corresponding filename
  for (i in seq_along(plots)) {
    save_plot(plots[[i]], plot_filenames[i])
  }
}


## S psihiatri

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E25", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 9 ~ "zdravstvo",
    row_number() > 9 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() == 1 ~ "psihiater",
    row_number() %in% c(2:4, 10:12) ~ "psiholog",
    row_number() %in% c(5:8, 13:15) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Function to save plots with a specified filename
save_plot <- function(plot, filename, width = 8, height = 6, dpi = 300) {
  ggsave(filename = filename, plot = plot, width = width, height = height, dpi = dpi)
}

# Perform t-tests and ANOVA if criteria met (normality and variance)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)

  # Post-hoc tests for ANOVA
  posthoc_min <- TukeyHSD(anova_min)
  posthoc_max <- TukeyHSD(anova_max)
  posthoc_avg <- TukeyHSD(anova_avg)
  posthoc_sd <- TukeyHSD(anova_sd)

  # Print ANOVA and post-hoc results
  print(summary(anova_min)); print(posthoc_min)
  print(summary(anova_max)); print(posthoc_max)
  print(summary(anova_avg)); print(posthoc_avg)
  print(summary(anova_sd)); print(posthoc_sd)

  # List of plots with titles
  plots <- list(
    ggplot(data, aes(x = Group, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by zdravstvo and soc. varstvo (p =", round(t_min$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by zdravstvo and soc. varstvo (p =", round(t_max$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by zdravstvo and soc. varstvo (p =", round(t_avg$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by zdravstvo and soc. varstvo (p =", round(t_sd$p.value, 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by poklic (ANOVA p =", round(summary(anova_min)[[1]][["Pr(>F)"]][1], 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by poklic (ANOVA p =", round(summary(anova_max)[[1]][["Pr(>F)"]][1], 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by poklic (ANOVA p =", round(summary(anova_avg)[[1]][["Pr(>F)"]][1], 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by poklic (ANOVA p =", round(summary(anova_sd)[[1]][["Pr(>F)"]][1], 4), ")"))
  )
  
  # Filenames for each plot
  plot_filenames <- c("Min_by_zdravstvo_and_socvarstvo.png", "Max_by_zdravstvo_and_socvarstvo.png", "Avg_by_zdravstvo_and_socvarstvo.png", "SD_by_zdravstvo_and_socvarstvo.png",
                      "Min_by_poklic.png", "Max_by_poklic.png", "Avg_by_poklic.png", "SD_by_poklic.png")
  
  # Save each plot with corresponding filename
  for (i in seq_along(plots)) {
    save_plot(plots[[i]], plot_filenames[i])
  }
}


In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E25", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define the groups for analysis based on row ranges
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 10 ~ "zdravstvo",
    row_number() > 10 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() == 2 ~ "psihiater",
    row_number() %in% c(3:5, 11:13) ~ "psiholog",
    row_number() %in% c(6:9, 14:16) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Check criteria for t-test and ANOVA
# Check normality and variance equality (Shapiro-Wilk for normality and Levene's test for variance)

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>% 
  group_by(Group) %>% 
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Perform t-tests and ANOVA if criteria met (normality p > 0.05 and homogeneity p > 0.05)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons (A, B, C, D)
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)
  
  # Post-hoc tests for ANOVA
  posthoc_min <- TukeyHSD(anova_min)
  posthoc_max <- TukeyHSD(anova_max)
  posthoc_avg <- TukeyHSD(anova_avg)
  posthoc_sd <- TukeyHSD(anova_sd)
  
  # Display ANOVA and post-hoc results
  print(summary(anova_min))
  print(posthoc_min)
  print(summary(anova_max))
  print(posthoc_max)
  print(summary(anova_avg))
  print(posthoc_avg)
  print(summary(anova_sd))
  print(posthoc_sd)
  
  # Adding p-values to group boxplots
  ggplot(data, aes(x = Group, y = Min)) + 
    geom_boxplot() + 
    labs(title = paste("Min by zdravstvo and soc. varstvo (p =", round(t_min$p.value, 4), ")"))

  ggplot(data, aes(x = Group, y = Max)) + 
    geom_boxplot() + 
    labs(title = paste("Max by zdravstvo and soc. varstvo (p =", round(t_max$p.value, 4), ")"))

  ggplot(data, aes(x = Group, y = Avg)) + 
    geom_boxplot() + 
    labs(title = paste("Avg by zdravstvo and soc. varstvo (p =", round(t_avg$p.value, 4), ")"))

  ggplot(data, aes(x = Group, y = SD)) + 
    geom_boxplot() + 
    labs(title = paste("SD by zdravstvo and soc. varstvo (p =", round(t_sd$p.value, 4), ")"))

  # Boxplots for SubGroup comparisons (with ANOVA)
  ggplot(data, aes(x = SubGroup, y = Min)) + 
    geom_boxplot() + 
    labs(title = paste("Min by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_min)[[1]][["Pr(>F)"]][1], 4), ")"))

  ggplot(data, aes(x = SubGroup, y = Max)) + 
    geom_boxplot() + 
    labs(title = paste("Max by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_max)[[1]][["Pr(>F)"]][1], 4), ")"))

  ggplot(data, aes(x = SubGroup, y = Avg)) + 
    geom_boxplot() + 
    labs(title = paste("Avg by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_avg)[[1]][["Pr(>F)"]][1], 4), ")"))

  ggplot(data, aes(x = SubGroup, y = SD)) + 
    geom_boxplot() + 
    labs(title = paste("SD by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_sd)[[1]][["Pr(>F)"]][1], 4), ")"))
}


In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E25", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define the groups for analysis based on row ranges
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 10 ~ "zdravstvo",
    row_number() > 10 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() == 2 ~ "psihiater",
    row_number() %in% c(3:5, 11:13) ~ "psiholog",
    row_number() %in% c(6:9, 14:16) ~ "klin_psiholog",
    TRUE ~ "delov_terap"
  ))

# Check criteria for t-test and ANOVA
# Check normality and variance equality (assuming Shapiro-Wilk for normality and Levene's test for variance)

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>% 
  group_by(Group) %>% 
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances (install 'car' package if needed)
library(car)
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Perform t-tests and ANOVA if criteria met (normality p > 0.05 and homogeneity p > 0.05)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons (A, B, C, D)
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)

  # Display results
  print(summary(anova_min))
  print(summary(anova_max))
  print(summary(anova_avg))
  print(summary(anova_sd))
}

# Plotting the results
# Boxplots for each measure by Group and SubGroup
ggplot(data, aes(x = Group, y = Min)) + geom_boxplot() + labs(title = "Min by zdravstvo and soc. varstvo")
ggplot(data, aes(x = Group, y = Max)) + geom_boxplot() + labs(title = "Max by zdravstvo and soc. varstvo")
ggplot(data, aes(x = Group, y = Avg)) + geom_boxplot() + labs(title = "Avg by zdravstvo and soc. varstvo")
ggplot(data, aes(x = Group, y = SD)) + geom_boxplot() + labs(title = "SD by zdravstvo and soc. varstvo")

ggplot(data, aes(x = SubGroup, y = Min)) + geom_boxplot() + labs(title = "Min by psihiater, psiholog, klin_psih, delov_terapevt")
ggplot(data, aes(x = SubGroup, y = Max)) + geom_boxplot() + labs(title = "Max by psihiater, psiholog, klin_psih, delov_terapevt")
ggplot(data, aes(x = SubGroup, y = Avg)) + geom_boxplot() + labs(title = "Avg by psihiater, psiholog, klin_psih, delov_terapevt")
ggplot(data, aes(x = SubGroup, y = SD)) + geom_boxplot() + labs(title = "SD by psihiater, psiholog, klin_psih, delov_terapevt")
